# Prezentacja 3

In [16]:
from pandas import read_csv, to_datetime, read_excel, Series, date_range
import numpy as np
from scipy.stats import t, norm, chi2
from collections import Counter
from arch import arch_model
from typing import Literal

###  Notowania giełdowe PEPSICO (PEP) - od 13.04.2015 do 31.03.2025, co tydzień

https://www.nasdaq.com/market-activity/stocks/pep/historical?page=1&rows_per_page=10&timeline=y10

In [7]:
dane_notowania = read_csv('dane\\notowania.csv')
dane_notowania['Date'] = to_datetime(dane_notowania['Date'], format='%m/%d/%Y')
dane_notowania = dane_notowania.sort_values('Date')
dane_notowania.set_index('Date', inplace=True)
# Ustalenie startowej daty: 13.04.2015
start_date = to_datetime('2015-05-10')
dane_notowania = dane_notowania[dane_notowania.index >= start_date]
dane_notowania = dane_notowania.resample('7D').first()
daty_notowania = dane_notowania.index.tolist()
dane_notowania = dane_notowania["Close/Last"].tolist()
dane_notowania = [float(x.replace('$', '')) for x in dane_notowania]
# aby uniknąć problemu związanemu z brakiem danych w weekendy i dni niehandlowe, bierzemy notowanie z każdego poniedziałku (jeśli nie ma z danego poniedziałku, to z wtorku etc.)

## VaR dla notowań

### Metoda parametryczna

In [8]:
# Obliczanie tygodniowych zmian procentowych
zmiany_procentowe = Series(dane_notowania).pct_change().dropna() * 100
daty_zmian = daty_notowania[1:]  # pierwszy tydzień nie ma zmiany

# straty
L = -np.array(zmiany_procentowe)

# Dopasuj rozkład t: zwraca df, loc, scale
df_fit, loc_fit, scale_fit = t.fit(L)

print("VaR rzędu 95% metodą parametryczną:", t.ppf(0.95, df_fit, loc=loc_fit, scale=scale_fit))
print("VaR rzędu 99% metodą parametryczną:", t.ppf(0.99, df_fit, loc=loc_fit, scale=scale_fit))

VaR rzędu 95% metodą parametryczną: 3.514487206967012
VaR rzędu 99% metodą parametryczną: 5.876445890530571


### Metoda historyczna (zwykła)

In [9]:
print("VaR rzędu 95% metodą historyczną (zwykłą):", np.quantile(L, 0.95))
print("VaR rzędu 99% metodą historyczną (zwykłą):", np.quantile(L, 0.99))

VaR rzędu 95% metodą historyczną (zwykłą): 3.5473920358945366
VaR rzędu 99% metodą historyczną (zwykłą): 5.6245881905998685


In [10]:
def weighted_historical_VaR(data, alpha, lambd=0.999):
    n = len(data)
    w_1 = (1 - lambd) / (1 - np.power(lambd, n))
    weights = np.power(lambd, n - np.arange(1, n + 1)) * w_1
    return np.quantile(data, alpha, weights=weights, method="inverted_cdf")

weights = np.arange(0.95, 0.99, 0.01)
for w in weights:
    print(f"VaR rzędu 95% metodą historyczną ważoną z wagą {w}:", weighted_historical_VaR(L, 0.95, w))
    print(f"VaR rzędu 99% metodą historyczną ważoną z wagą {w}:", weighted_historical_VaR(L, 0.99, w))

VaR rzędu 95% metodą historyczną ważoną z wagą 0.95: 4.501800720288118
VaR rzędu 99% metodą historyczną ważoną z wagą 0.95: 5.623368376490512
VaR rzędu 95% metodą historyczną ważoną z wagą 0.96: 4.501800720288118
VaR rzędu 99% metodą historyczną ważoną z wagą 0.96: 5.623368376490512
VaR rzędu 95% metodą historyczną ważoną z wagą 0.97: 4.501800720288118
VaR rzędu 99% metodą historyczną ważoną z wagą 0.97: 5.623368376490512
VaR rzędu 95% metodą historyczną ważoną z wagą 0.98: 3.9395754308910735
VaR rzędu 99% metodą historyczną ważoną z wagą 0.98: 5.623368376490512
VaR rzędu 95% metodą historyczną ważoną z wagą 0.99: 3.843380844101829
VaR rzędu 99% metodą historyczną ważoną z wagą 0.99: 5.623368376490512


### VaR obliczony z użyciem filtrowania szeregiem GARCH

In [14]:
def garch_VaR(data, alpha, dist: Literal['Normal', 't'] = 'Normal'):
    model = arch_model(data, vol='Garch', p=1, q=1, dist=dist)
    res = model.fit(disp='off')

    mu_hat = res.params['mu']
    sigma_t = res.conditional_volatility

    standardized_residuals = (data - mu_hat) / sigma_t

    q_alpha = np.quantile(standardized_residuals, alpha)

    forecast = res.forecast(horizon=1)
    sigma_n1 = np.sqrt(forecast.variance.values[-1, :])[0]

    VaR_alpha = sigma_n1 * q_alpha + mu_hat
    return VaR_alpha

print(f"VaR na poziomie 95% z użyciem filtrowania szeregiem GARCH: {garch_VaR(L, 0.95)}")
print(f"VaR na poziomie 99% z użyciem filtrowania szeregiem GARCH: {garch_VaR(L, 0.99)}")

VaR na poziomie 95% z użyciem filtrowania szeregiem GARCH: 5.043244842559927
VaR na poziomie 99% z użyciem filtrowania szeregiem GARCH: 7.4926245431342835


In [15]:
dane_cukier = [
    2.55,2.52,2.45,2.45,2.41,2.38,2.33,2.69,2.89,2.86,2.77,2.86, # 2010
    3.01,3.43,4.71,4.72,4.33,3.93,3.93,3.61,3.68,3.61,3.72,3.61, # 2011
    3.78,3.79,3.79,3.84,3.80,3.84,3.80,3.77,3.78,3.73,3.78,3.76, # 2012
    3.67,3.53,3.67,3.55,3.55,3.47,3.33,3.22,3.14,3.15,3.01,2.85, # 2013
    2.83,2.73,2.58,2.35,2.18,2.24,2.04,2.16,1.93,1.98,1.89,1.81, # 2014
    1.85,1.85,1.98,2.00,1.98,1.94,2.20,2.21,2.30,2.28,2.33,2.34, # 2015
    2.32,2.55,2.60,2.61,2.67,2.65,2.91,2.82,3.07,2.96,3.11,2.98, # 2016
    3.16,2.95,2.93,3.14,2.97,3.12,2.97,3.06,2.84,2.71,2.87,2.46, # 2017
    2.02,2.48,1.96,2.41,1.96,1.91,2.14,2.00,2.03,2.21,2.06,1.87, # 2018
    2.36,2.36,2.57,2.34,2.65,2.37,2.62,2.47,2.68,2.42,2.73,2.42, # 2019
    2.70,2.50,2.54,2.63,2.72,2.66,2.47,2.65,2.38,2.38,2.59,2.57, # 2020
    2.75,2.42,2.53,2.81,2.61,2.51,3.01,2.44,2.53,3.11,3.03,3.33, # 2021
    3.06,3.06,3.33,3.34,3.93,3.33,4.25,5.49,6.76,5.65,6.64,6.00, # 2022
    6.68,6.02,6.63,6.02,6.59,5.90,6.54,5.81,6.14,5.60,6.03,5.70, # 2023
    6.12,4.76,4.90,5.09,4.55,5.31,4.40,4.86,3.65,4.53,3.51,3.30] # 2024

# Obliczanie tygodniowych zmian procentowych
zmiany_procentowe = Series(dane_cukier).pct_change().dropna() * 100

Lc = -np.array(zmiany_procentowe)

df_fit, loc_fit, scale_fit = t.fit(Lc)

print("VaR rzędu 95% metodą parametryczną:", t.ppf(0.95, df=df_fit, loc=loc_fit, scale=scale_fit))
print("VaR rzędu 99% metodą parametryczną:", t.ppf(0.99, df=df_fit, loc=loc_fit, scale=scale_fit))

print("VaR rzędu 95% metodą historyczną (zwykłą):", np.quantile(Lc, 0.95))
print("VaR rzędu 99% metodą historyczną (zwykłą):", np.quantile(Lc, 0.99))


print("VaR rzędu 95% metodą historyczną ważoną z wagą 0.97:",weighted_historical_VaR(Lc, 0.95, 0.97))
print("VaR rzędu 99% metodą historyczną ważoną z wagą 0.97:", weighted_historical_VaR(Lc, 0.99, 0.97))

print(f"VaR na poziomie 95% z użyciem filtrowania szeregiem GARCH: {garch_VaR(Lc, 0.95)}")
print(f"VaR na poziomie 99% z użyciem filtrowania szeregiem GARCH: {garch_VaR(Lc, 0.99)}")

VaR rzędu 95% metodą parametryczną: 16.041620735736437
VaR rzędu 99% metodą parametryczną: 27.47799656916375
VaR rzędu 95% metodą historyczną (zwykłą): 15.382469849586696
VaR rzędu 99% metodą historyczną (zwykłą): 22.2869757174393
VaR rzędu 95% metodą historyczną ważoną z wagą 0.97: 22.51655629139073
VaR rzędu 99% metodą historyczną ważoną z wagą 0.97: 24.89711934156379
VaR na poziomie 95% z użyciem filtrowania szeregiem GARCH: 19.322753138713622
VaR na poziomie 99% z użyciem filtrowania szeregiem GARCH: 28.211834246942875


## Testy dla wartości zagrożonej

### Test kupca

In [17]:
def backtesting(data, alpha, method: Literal['t', 'norm', 'historical', 'weighted', 'garch'] = 'historical'):
    I = []
    n = (len(data)) // 2
    for i in range((len(data) + 1) // 2):
        if method == 'historical':
            q = np.quantile(data[i:i + n], 1-alpha)
        elif method == 'norm':
            loc_fit, scale_fit = norm.fit(data[i:i + n])
            q = norm.ppf(1-alpha, loc=loc_fit, scale=scale_fit)
        elif method == 't':
            df_fit, loc_fit, scale_fit = t.fit(data[i:i + n])
            q = t.ppf(1-alpha, df=df_fit, loc=loc_fit, scale=scale_fit)
        elif method == 'weighted':
            q = weighted_historical_VaR(data[i:i + n], 1-alpha)
        elif method == 'garch':
            q = garch_VaR(data=data[i:i + n], alpha=1-alpha)
        else:
            raise ValueError('Niepoprawnie zadana metoda. Możliwe warianty: \'t\', \'norm\', \'historical\', \'weighted\', \'garch\'')
        if data[i+n] >= q:
            I.append(1)
        else:
            I.append(0)
    return I


def unconditional_coverage_test(data, alpha, method: Literal['t', 'norm', 'historical', 'weighted', 'garch'] = 'historical'):
    I = backtesting(data, alpha, method=method)
    i1 = np.sum(I)
    i0 = len(I) - i1

    p0 = np.mean(I)
    LR = - np.log(((1-alpha) ** i0 * alpha ** i1) / ((1 - p0) ** i0 * p0 ** i1))
    p_value = chi2.cdf(LR, df=1)
    return p_value

In [19]:
VaR_method = ['t', 'historical', 'weighted', 'garch']
for method in VaR_method:
    print(f"P-wartość w teście kupca dla metody {method}:", unconditional_coverage_test(L, 0.05, method=method))

P-wartość w teście kupca dla metody t: 0.14963736900288463
P-wartość w teście kupca dla metody historical: 0.552454256813111
P-wartość w teście kupca dla metody weighted: 0.4334139868202683
P-wartość w teście kupca dla metody garch: 0.2982125345113053


### Test niezależności (Christoffersena) 

In [20]:
def christoffersen_test(data, alpha, method: Literal['t', 'norm', 'historical', 'weighted', 'garch'] = 'historical'):
    I_alpha = backtesting(data, alpha, method=method)       # wektor 0 i 1 z rozkładu wielomianowego z pr sukcesu 1
    I = list(zip(I_alpha[:-1], I_alpha[1:]))
    I = Counter(I)                                          # słownik z listą wystąpień (0, 0), (0, 1), (1, 0) i (1, 1)
    pi = np.matrix([[I[(0, 0)] / (I[(0, 0)] + I[(0, 1)]), I[(0, 1)] / (I[(0, 0)] + I[(0, 1)])],
                    [I[(1, 0)] / (I[(1, 0)] + I[(1, 1)]), I[(1, 1)] / (I[(1, 0)] + I[(1, 1)])],])

    pi_est = np.mean(I_alpha)

    PI_est = np.matrix([[1 - pi_est, pi_est], [1 - pi_est, pi_est]])

    def L(pi: np.matrix):
        return ((1 - pi[0, 1]) ** I[(0, 0)] * pi[0, 1] ** I[(0, 1)] *
                (1 - pi[1, 1]) ** I[(1, 0)] * pi[1, 1] ** I[(1, 1)])

    LR = - np.log(L(PI_est) / L(pi))
    p_value = chi2.cdf(LR, df=1)
    return p_value

In [21]:
VaR_method = ['t', 'historical', 'weighted', 'garch']
for method in VaR_method:
    print(f"P-wartość w teście christoffersena dla metody {method}:", christoffersen_test(L, 0.05, method=method))

P-wartość w teście christoffersena dla metody t: 0.6280943923820427
P-wartość w teście christoffersena dla metody historical: 0.7247523761356631
P-wartość w teście christoffersena dla metody weighted: 0.6945064920072925
P-wartość w teście christoffersena dla metody garch: 0.6622778487364459
